In [4]:
# Library imports
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [5]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [6]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

In [7]:
# assigning values unto text
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 40
STEP = 3

sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

In [9]:
# putting training data into a numpy array
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_characters[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.fit(x, y, batch_size=256, epochs=4)
model.save('textgenerator.model')

Epoch 1/4
651/651 [==============================] - 118s 178ms/step - loss: 2.7158
Epoch 2/4
651/651 [==============================] - 115s 177ms/step - loss: 2.3169
Epoch 3/4
651/651 [==============================] - 117s 179ms/step - loss: 2.2043
Epoch 4/4
651/651 [==============================] - 117s 179ms/step - loss: 2.1283


In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated_text = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated_text += sentence
  for i in range(length):
      x_pred = np.zeros((1, SEQ_LENGTH, len(characters)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_to_index[char]] = 1.

      predictions = model.predict(x_pred, verbose=0)[0]
      next_index = sample(predictions, temperature)
      next_char = index_to_char[next_index]

      generated_text += next_char
      sentence = sentence[1:] + next_char

  return generated_text


print("------0.2--------")
print(generate_text(300, 0.2))
print("------0.6--------")
print(generate_text(300, 0.6))
print("------0.5--------")
print(generate_text(300, 0.5))
print("------0.8--------")
print(generate_text(300, 0.8))
print("------1.0--------")
print(generate_text(300, 1.0))

------0.2--------
d,
i dare your quenchless fury to more roust and hard and the hard and the hare the here the her the the beather in the hard and the hard and and the hard and and and and the hare her here the hard the hare the hare the hand and the hand the her the hing the hand the hare so the the hour the hath the hand the hare the and the hare the her
------0.6--------
,
she came adorned hither like sweet may that to hons wich to the peat thobes then, for, the e the ke the porsto for couns and the pare,
i me thil the sartse the farke the well thoun were stay, harce on the king som,
and hene ho hear heant thie the ater,
shave fores nos couns and we tha his shand chand i whe hard.

henme the kay have in t
------0.5--------
gonus:
hang all the husbands
that cannot hane the wath the wing the forlly grather, and hear hear lof hear my mens be where ol my king the land the the are in the pared whin the fare,
and dow he be the preather ware stoun the here is himerenes mone the gand her an